# Part I. ETL Pipeline for Pre-Processing the Files

In this section we will work in the pre processing of the files.

## Importing packages

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

## Importing files

Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


Check the resulting list

In [3]:
file_path_list

['/workspace/home/event_data/2018-11-08-events.csv',
 '/workspace/home/event_data/2018-11-06-events.csv',
 '/workspace/home/event_data/2018-11-17-events.csv',
 '/workspace/home/event_data/2018-11-02-events.csv',
 '/workspace/home/event_data/2018-11-10-events.csv',
 '/workspace/home/event_data/2018-11-14-events.csv',
 '/workspace/home/event_data/2018-11-07-events.csv',
 '/workspace/home/event_data/2018-11-19-events.csv',
 '/workspace/home/event_data/2018-11-23-events.csv',
 '/workspace/home/event_data/2018-11-24-events.csv',
 '/workspace/home/event_data/2018-11-22-events.csv',
 '/workspace/home/event_data/2018-11-26-events.csv',
 '/workspace/home/event_data/2018-11-09-events.csv',
 '/workspace/home/event_data/2018-11-04-events.csv',
 '/workspace/home/event_data/2018-11-30-events.csv',
 '/workspace/home/event_data/2018-11-12-events.csv',
 '/workspace/home/event_data/2018-11-25-events.csv',
 '/workspace/home/event_data/2018-11-29-events.csv',
 '/workspace/home/event_data/2018-11-03-events

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:
# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# Get total number of rows 
print(len(full_data_rows_list))

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    # There are 6821 rows including the header.
    print(sum(1 for line in f))

# print the rows to check the data
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    csvreader = csv.reader(f) 
    for i, line in enumerate(csvreader):
        print(line)
        

6821
['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length', 'level', 'location', 'sessionId', 'song', 'userId']
['Slipknot', 'Aiden', 'M', '0', 'Ramirez', '192.57424', 'paid', 'New York-Newark-Jersey City, NY-NJ-PA', '19', 'Opium Of The People (Album Version)', '20']
['Cat Stevens', 'Aiden', 'M', '1', 'Ramirez', '170.57914', 'paid', 'New York-Newark-Jersey City, NY-NJ-PA', '19', "Don't Be Shy", '20']
['Collective Soul', 'Aiden', 'M', '2', 'Ramirez', '273.47546', 'paid', 'New York-Newark-Jersey City, NY-NJ-PA', '19', 'Run (LP Version)', '20']
['Taylor Swift', 'Aiden', 'M', '3', 'Ramirez', '233.89995', 'paid', 'New York-Newark-Jersey City, NY-NJ-PA', '19', 'Love Story', '20']
['The Dixie Cups', 'Aiden', 'M', '4', 'Ramirez', '120.05832', 'paid', 'New York-Newark-Jersey City, NY-NJ-PA', '19', 'Iko Iko', '20']
['Carlos Santana & Mahavishnu John McLaughlin', 'Aiden', 'M', '5', 'Ramirez', '157.93587', 'paid', 'New York-Newark-Jersey City, NY-NJ-PA', '19', 'Meditation', '20']

['Hardline', 'Matthew', 'M', '4', 'Jones', '234.73587', 'paid', 'Janesville-Beloit, WI', '439', 'Everything', '36']
['The Funky Lowlives', 'Matthew', 'M', '5', 'Jones', '280.34567', 'paid', 'Janesville-Beloit, WI', '439', 'Sail Into the Sun', '36']
['DL Incognito', 'Matthew', 'M', '6', 'Jones', '221.07383', 'paid', 'Janesville-Beloit, WI', '439', 'Proof', '36']
['Justice', 'Matthew', 'M', '7', 'Jones', '243.40853', 'paid', 'Janesville-Beloit, WI', '439', 'TTHHEE PPAARRTTYY', '36']
['Earth_ Wind & Fire', 'Theodore', 'M', '1', 'Smith', '178.20689', 'free', 'Houston-The Woodlands-Sugar Land, TX', '447', "Night Dreamin'", '52']
['Strawbs', 'Matthew', 'M', '8', 'Jones', '255.81669', 'paid', 'Janesville-Beloit, WI', '439', 'Sheep', '36']
['Angus & Julia Stone', 'Matthew', 'M', '9', 'Jones', '172.85179', 'paid', 'Janesville-Beloit, WI', '439', 'Wasted', '36']
['Sara Bareilles', 'Matthew', 'M', '10', 'Jones', '260.8322', 'paid', 'Janesville-Beloit, WI', '439', 'Love Song', '36']
['Bruna Caram'

['Chevelle', 'Chloe', 'F', '3', 'Cuevas', '250.06975', 'paid', 'San Francisco-Oakland-Hayward, CA', '708', "Wonder What's Next", '49']
['The Cinematic Orchestra', 'Chloe', 'F', '4', 'Cuevas', '373.9424', 'paid', 'San Francisco-Oakland-Hayward, CA', '708', 'To Build A Home', '49']
['Between The Buried And Me', 'Chloe', 'F', '5', 'Cuevas', '217.75628', 'paid', 'San Francisco-Oakland-Hayward, CA', '708', 'Mirrors', '49']
['Kealii Reichel', 'Chloe', 'F', '6', 'Cuevas', '287.65995', 'paid', 'San Francisco-Oakland-Hayward, CA', '708', 'Kawaipunahele (Album Version)', '49']
['Tub Ring', 'Chloe', 'F', '7', 'Cuevas', '233.69098', 'paid', 'San Francisco-Oakland-Hayward, CA', '708', 'Invalid', '49']
['Joe Christmas', 'Chloe', 'F', '8', 'Cuevas', '185.7824', 'paid', 'San Francisco-Oakland-Hayward, CA', '708', 'Bedroom Suite', '49']
['Die Mooskirchner', 'Chloe', 'F', '9', 'Cuevas', '169.29914', 'paid', 'San Francisco-Oakland-Hayward, CA', '708', "Frisch und g'sund", '49']
['Devendra Banhart', 'Chlo

['Kate Voegele', 'Layla', 'F', '8', 'Griffin', '218.01751', 'paid', 'Lake Havasu City-Kingman, AZ', '809', "You Can't Break A Broken Heart", '24']
['Bill Withers', 'Avery', 'F', '14', 'Watkins', '383.73832', 'paid', 'San Jose-Sunnyvale-Santa Clara, CA', '691', 'Make Love To Your Mind', '30']
['Matt Redman', 'Sylvie', 'F', '0', 'Cruz', '303.46404', 'free', 'Washington-Arlington-Alexandria, DC-VA-MD-WV', '864', 'Blessed Be Your Name', '10']
['Stacie Orrico', 'Celeste', 'F', '1', 'Williams', '225.35791', 'free', 'Klamath Falls, OR', '860', 'Stuck (Album Version)', '53']
['Shadows Fall', 'Layla', 'F', '9', 'Griffin', '220.3424', 'paid', 'Lake Havasu City-Kingman, AZ', '809', 'War', '24']
['The Killers', 'Avery', 'F', '16', 'Watkins', '246.80444', 'paid', 'San Jose-Sunnyvale-Santa Clara, CA', '691', 'Read My Mind', '30']
['Atmosphere', 'Avery', 'F', '17', 'Watkins', '257.38404', 'paid', 'San Jose-Sunnyvale-Santa Clara, CA', '691', 'Trying To Find A Balance', '30']
['La Ley', 'Emily', 'F', '

['Cherise', 'Stefany', 'F', '0', 'White', '229.69424', 'free', 'Lubbock, TX', '772', 'No Good 4 You', '83']
['Anna Waronker', 'Jayden', 'F', '0', 'Duffy', '189.6224', 'free', 'Seattle-Tacoma-Bellevue, WA', '662', 'Nothing Personal', '76']
['King ChangÃ\x83Â³', 'Cecilia', 'F', '0', 'Owens', '340.74077', 'free', 'Atlanta-Sandy Springs-Roswell, GA', '763', 'ConfesiÃ\x83Â³n', '6']
['Gang Of Four', 'Cecilia', 'F', '1', 'Owens', '193.14893', 'free', 'Atlanta-Sandy Springs-Roswell, GA', '763', 'I Found That Essence Rare', '6']
['Line Renaud', 'Cecilia', 'F', '2', 'Owens', '176.16934', 'free', 'Atlanta-Sandy Springs-Roswell, GA', '763', 'Le Soir', '6']
['the bird and the bee', 'Tegan', 'F', '2', 'Levine', '198.1122', 'paid', 'Portland-South Portland, ME', '823', 'Last Day Of Our Love', '80']
['Muse', 'Jordan', 'F', '3', 'Hicks', '259.26485', 'free', 'Salinas, CA', '814', 'Supermassive Black Hole [Phones Control Voltage Remix]', '37']
['Mobin Master Feat. Robin S.', 'Jayden', 'M', '1', 'Fox', '

...

['Kings Of Leon', 'Jacqueline', 'F', '23', 'Lynch', '227.18649', 'paid', 'Atlanta-Sandy Springs-Roswell, GA', '486', 'Be Somebody', '29']
['Cornelis Vreeswijk', 'Jacqueline', 'F', '24', 'Lynch', '140.66893', 'paid', 'Atlanta-Sandy Springs-Roswell, GA', '486', 'En resa', '29']
['Bon Iver', 'Jordan', 'F', '2', 'Hicks', '238.49751', 'free', 'Salinas, CA', '499', 'Skinny Love', '37']
['Daft Punk', 'Jacqueline', 'F', '25', 'Lynch', '290.76853', 'paid', 'Atlanta-Sandy Springs-Roswell, GA', '486', 'Television Rules The Nation / Crescendolls', '29']
['Asher Roth / Cee-Lo', 'Jordan', 'F', '3', 'Hicks', '262.66077', 'free', 'Salinas, CA', '499', 'Be By Myself', '37']
['Suicidal Tendencies', 'Jacqueline', 'F', '26', 'Lynch', '172.01587', 'paid', 'Atlanta-Sandy Springs-Roswell, GA', '486', 'Freedumb', '29']
['The Be Good Tanyas', 'Jordan', 'F', '4', 'Hicks', '225.04444', 'free', 'Salinas, CA', '499', 'A Thousand Tiny Pieces', '37']
['John Mayer', 'Jacqueline', 'F', '27', 'Lynch', '250.38322', 'pai

['Incubus', 'Chloe', 'F', '15', 'Cuevas', '229.22404', 'paid', 'San Francisco-Oakland-Hayward, CA', '636', 'Pardon Me', '49']
['Edwyn Collins', 'Tegan', 'F', '9', 'Levine', '235.91138', 'paid', 'Portland-South Portland, ME', '620', 'A Girl Like You', '80']
['Joshua Radin', 'Chloe', 'F', '16', 'Cuevas', '182.62159', 'paid', 'San Francisco-Oakland-Hayward, CA', '636', "The Fear You Won't Fall", '49']
['Bon Iver', 'Chloe', 'F', '17', 'Cuevas', '116.89751', 'paid', 'San Francisco-Oakland-Hayward, CA', '636', 'Team', '49']
['Nipsey Hussle', 'Chloe', 'F', '18', 'Cuevas', '170.34404', 'paid', 'San Francisco-Oakland-Hayward, CA', '636', 'Its Hard Out Here', '49']
['Bebo & Cigala', 'Tegan', 'F', '11', 'Levine', '331.15383', 'paid', 'Portland-South Portland, ME', '620', 'Lagrimas Negras', '80']
['Kings Of Leon', 'Tegan', 'F', '12', 'Levine', '201.79546', 'paid', 'Portland-South Portland, ME', '620', 'Revelry', '80']
['Amy Winehouse', 'Tegan', 'F', '13', 'Levine', '223.9473', 'paid', 'Portland-So

['RÃ\x83Â¶yksopp', 'Layla', 'F', '42', 'Griffin', '210.99057', 'paid', 'Lake Havasu City-Kingman, AZ', '984', 'In Space', '24']
['Sounds from the Ground', 'Jacob', 'M', '11', 'Klein', '458.31791', 'paid', 'Tampa-St. Petersburg-Clearwater, FL', '954', 'Where The Wild Things Were', '73']
['Mantronix', 'Layla', 'F', '43', 'Griffin', '353.64526', 'paid', 'Lake Havasu City-Kingman, AZ', '984', 'Ladies (Dub) (2005 Digital Remaster)', '24']
['Death Cab for Cutie', 'Layla', 'F', '44', 'Griffin', '189.3873', 'paid', 'Lake Havasu City-Kingman, AZ', '984', 'I Will Follow You into the Dark (Album Version)', '24']
['Green Day', 'Jacob', 'M', '12', 'Klein', '256.02567', 'paid', 'Tampa-St. Petersburg-Clearwater, FL', '954', 'Last Night On Earth [feat. Green Day & The Cast Of American Idiot] (Album Version)', '73']
['Scooter', 'Layla', 'F', '45', 'Griffin', '347.19302', 'paid', 'Lake Havasu City-Kingman, AZ', '984', 'Across The Sky', '24']
['Bodo Wartke', 'Ryan', 'M', '0', 'Smith', '377.41669', 'free'

['The Newbeats', 'Harper', 'M', '5', 'Barrett', '109.24363', 'paid', 'New York-Newark-Jersey City, NY-NJ-PA', '404', 'Bread And Butter', '42']
['Kings Of Leon', 'Kate', 'F', '43', 'Harrell', '260.38812', 'paid', 'Lansing-East Lansing, MI', '605', 'Dusty', '97']
['Los Lobos', 'Avery', 'F', '110', 'Watkins', '174.18404', 'paid', 'San Jose-Sunnyvale-Santa Clara, CA', '324', 'La Bamba', '30']
['Drake / Kanye West / Lil Wayne / Eminem', 'Harper', 'M', '7', 'Barrett', '357.66812', 'paid', 'New York-Newark-Jersey City, NY-NJ-PA', '404', 'Forever', '42']
['Ginuwine', 'Chloe', 'F', '10', 'Cuevas', '244.27057', 'paid', 'San Francisco-Oakland-Hayward, CA', '621', "I'm In Love", '49']
['Descendents', 'Avery', 'F', '111', 'Watkins', '89.23383', 'paid', 'San Jose-Sunnyvale-Santa Clara, CA', '324', "I'm Not A Loser", '30']
['Jennifer Love Hewitt', 'Kate', 'F', '44', 'Harrell', '222.17098', 'paid', 'Lansing-East Lansing, MI', '605', 'BareNaked', '97']
['The Cinematic Orchestra', 'Chloe', 'F', '11', 'C

['Tevin Campbell', 'Tegan', 'F', '28', 'Levine', '286.69342', 'paid', 'Portland-South Portland, ME', '774', "I'm Ready (Album Version)", '80']
['Hector Lavoe', 'Tegan', 'F', '29', 'Levine', '330.70975', 'paid', 'Portland-South Portland, ME', '774', 'Hacha Y Machete', '80']
['Alliance Ethnik', 'Tegan', 'F', '30', 'Levine', '195.94404', 'paid', 'Portland-South Portland, ME', '774', 'SinceritÃ\x83Â© Et Jalousie', '80']
['OneRepublic', 'Tegan', 'F', '31', 'Levine', '224.67873', 'paid', 'Portland-South Portland, ME', '774', 'Secrets', '80']
['Marea', 'Tegan', 'F', '32', 'Levine', '261.90322', 'paid', 'Portland-South Portland, ME', '774', 'Marea (version acustica)', '80']
['Mutemath', 'Tegan', 'F', '33', 'Levine', '72.95955', 'paid', 'Portland-South Portland, ME', '774', 'Collapse (Album Version)', '80']
['Yeasayer', 'Tegan', 'F', '34', 'Levine', '241.89342', 'paid', 'Portland-South Portland, ME', '774', 'Ambling Alp (DJ /Rupture & Brent Arnold Remix)', '80']
['Righteous Brothers', 'Tegan', 

We can see that the first row is the header of the table. Later, we will be checking the number of rows of the tables we build.

# Part II. Apache Cassandra. 

Now we are ready to work with the CSV file titled event_Data_file_new.csv. This file has the following columns:
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the ETL has been run:<br>

<img src="images/image_event_datafile_new.jpg">

## Previous steps

### Creating a Cluster

In [6]:
# We make a connection to a Cassandra instance in local machine (127.0.0.1)
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# We create a session
session = cluster.connect()

### Create Keyspace

In [7]:
# Create a Keyspace 
session.execute("CREATE KEYSPACE IF NOT EXISTS music_keyspace WITH replication = \
                {'class': 'SimpleStrategy', 'replication_factor': '1'}")

### Set Keyspace

In [8]:
# We set the keyspace to the created one
session.set_keyspace('music_keyspace')

## Modeling

Now we will create tables to run the following queries. With Apache Cassandra we model the database tables on the queries we want to run.

## We will model three questions of the data.

1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Query 1
Artist, song title and song's lenght that was heard during session 338 and item 4.

In this case, our data will be partitioned by session and we will use itemInSession as a clustering key, so we can use those columns in the where clause. Also, we will add columns artist, song and lenght. 

In [9]:
# Drop table in case it already exists
session.execute('Drop table if exists SessionInfo')

In [10]:
## We create the table SessionInfo with the columns mentioned before.
query = """CREATE TABLE IF NOT EXISTS SessionInfo (
                                SessionId int,
                                ItemInSession int,
                                artist text,
                                song text,
                                lenght decimal,
                                primary key(SessionId, ItemInSession))"""
session.execute(query)                    

In [11]:
# file is the path to the csv we sant to load.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## We generate the Insert statments.
        query = "Insert into SessionInfo (SessionId, ItemInSession, Artist, Song, Lenght)"
        query = query + "values (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [12]:
# We check the number of rows to see if the modeling was right.
a = session.execute('Select count(*) from SessionInfo')
a[0]

Row(count=6820)

In [13]:
## We verify the data was entered into the table
select = """Select artist, song, lenght 
            FROM SessionInfo
            where 
                sessionId=338
                and itemInSession=4"""

rows = session.execute(select)

# We show the pandas DataFrame with the query's result.
pd.DataFrame([row for row in rows])

,artist,song,lenght
0,Faithless,Music Matters (Mark Knight Dub),495.3073


### Query 2
Name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182.

In this case, we use user id and session id as partition key. We add Item in session as clustering key to sort the values. The Primary key will be UserId, SessionId and ItemInSession. We have to add artist and user first and last name.

In [14]:
# Drop table in case it already exists
session.execute('Drop table if exists SessionsByUser')

In [15]:
## We create the table for the second query.
query = """CREATE TABLE IF NOT EXISTS SessionsByUser (
                                userid int,
                                SessionId int,
                                ItemInSession int,
                                artist text,
                                song text,
                                firstname text,
                                lastname text,
                                primary key((userid, sessionid), ItemInSession))"""
session.execute(query)                

In [16]:
# we repeat the same steps as before
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "Insert into SessionsByUser (userid, SessionId, ItemInSession, Artist,Song,FirstName,LastName )"
        query = query + "values (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[-1]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4], ))

Again, we check the number of rows

In [17]:
a = session.execute('Select count(*) from SessionsByUser')
a[0]

Row(count=6820)

In [18]:
select = """Select 
                artist, song, firstname, lastname
            FROM SessionsByUser
            where 
                userid=10
                and sessionid=182"""

rows = session.execute(select)

# We show the pandas DataFrame with the query's result.
pd.DataFrame([row for row in rows])

,artist,song,firstname,lastname
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


# Query 3
Every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

We will use the song as partition key because we will need it in the where clause. We add user id as clustering column, so our primary key will be song and user id. We have to do this because two users' names can be the same, but still be different users. To model this, we will use that primary key. 

Also, we will see that this model will result in less rows. This happens because one single user can listen to the same song many times. We could add the item in session or the session id to the primary key, but in this case the number of times isn't really important.

In [19]:
# Drop table in case it already exists
session.execute('Drop table if exists UserAndSongs')

In [20]:
query = """CREATE TABLE IF NOT EXISTS UserAndSongs (
                                song text,
                                userid int,
                                firstname text,
                                lastname text,
                                primary key(song, userid))"""
session.execute(query)    

In [21]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "Insert into UserAndSongs (Song, userid, FirstName,LastName)"
        query = query + "values (%s, %s, %s, %s)"
        session.execute(query, (line[9] ,int(line[-1]), line[1], line[4]))

In [22]:
# We check the total number of rows. As we said, is less than the original number.
a = session.execute('Select count(*) from UserAndSongs')
a[0]

Row(count=6618)

In [23]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
select = """Select 
                firstname, lastname, userid
            FROM UserAndSongs
            where 
                song='All Hands Against His Own'"""

rows = session.execute(select)

# We show the pandas DataFrame with the query's result.
pd.DataFrame([row for row in rows])

,firstname,lastname,userid
0,Jacqueline,Lynch,29
1,Tegan,Levine,80
2,Sara,Johnson,95


### Drop the tables before closing out the sessions

In [24]:
list_of_tables = ['SessionInfo', 'SessionsByUser', 'UserAndSongs']

for t in list_of_tables:
    session.execute(f'Drop table {t}')

### Close the session and cluster connection¶

In [25]:
session.shutdown()
cluster.shutdown()